# Import

In [1]:
import os
from time import sleep
from time import time as unixtime
from typing import Callable, List
import random
from math import ceil
import yaml
import string
import json
from urllib.parse import urlparse
from time import sleep
import re
from operator import itemgetter
import subprocess
from tempfile import NamedTemporaryFile

from dotenv import load_dotenv
from IPython.display import display, clear_output

import boto3
import kubernetes
from kubernetes.client.rest import ApiException

from pyhelm3 import Client as HelmClient


In [2]:
load_dotenv('/.env')

True

In [3]:
from helper import wait_until
from helper.ec2 import (get_vpcs_ids, get_internet_gateway_ids_attached_to_vpc, 
                        get_route_table_ids_for_vpc, route_to_gateway_exists, 
                        get_subnet_ids_in_vpc, get_security_group_ids)
from helper.k8s import (get_one_running_pod, get_jupyter_token_from_pod)

# Define

In [4]:
REGION = 'ca-central-1'
CLUSTER_NAME = 'kubyterlab-llm'
EBS_VOLUME_SIZE = 500 # GiB
TAGS = {'cluster': CLUSTER_NAME, 'purpose': 'llm'}  # Do not change the keys, they are hardcoded throughout.
CLUSTER_TAGS = {'cluster': CLUSTER_NAME}
CLUSTER_FILTERS = [{'Name': f'tag:{k}', 'Values': [CLUSTER_TAGS[k]]} for k in CLUSTER_TAGS]
K8S_VERSION = os.environ['K8S_VERSION']  # '1.30'
K8S_VERSION = '.'.join(K8S_VERSION.split('.')[:2]) if len(K8S_VERSION.split('.')) > 2 else K8S_VERSION
INSTANCE_TYPES = {'gpu': ['g4dn.2xlarge'], 'default': ['t3.medium']}
ALLOWED_PORTS = [8888]

In [5]:
# CLUSTER_FILTERS

In [6]:
def check_all_vpcs_available(response: dict) -> True:
    if not 'Vpcs' in response:
        raise ValueError
    return all([vpc.get('State', '') == 'available' for vpc in response['Vpcs']])

In [7]:
def is_cluster_active(response: dict) -> bool:
    status = response['cluster']['status']
    clear_output(wait=True)
    display(status)
    return status == 'ACTIVE'


In [8]:
def is_node_group_active(response: dict) -> bool:
    status = response['nodegroup']['status']
    clear_output(wait=True)
    display(status)
    return status in ['ACTIVE', 'CREATE_FAILED']


In [9]:
def is_snapshot_completed(response: dict) -> bool:
    state = response['Snapshots'][0]['State']
    clear_output(wait=True)
    display(state)
    return state.lower() == 'completed'


In [10]:
def is_volume_available(response: dict) -> bool:
    state = response['Volumes'][0]['State']
    clear_output(wait=True)
    display(state)
    return state.lower() == 'available'


# Instantiate

In [11]:
session = boto3.Session(region_name=REGION)
eks_client = session.client('eks')
ec2_client = session.client('ec2')
iam_client = session.client('iam')

aws_account_id = boto3.client('sts').get_caller_identity().get('Account')

# Create or Restore Volume

In [12]:
try:
    response = ec2_client.describe_volumes(Filters=CLUSTER_FILTERS)
    volumes = response.get('Volumes', [])
except RuntimeError:
    volumes = []
volume_ids = [volume['VolumeId'] for volume in volumes]
volume_ids

[]

In [13]:
assert len(volume_ids) <= 1  # TODO: Get the latest one if more than one.
if volume_ids:
    volume_id = volume_ids[0]
    availability_zone = volumes[0]['AvailabilityZone']
else:
    volume_id = None
volume_id

In [14]:
if not volume_id:
    response = ec2_client.describe_snapshots(Filters=CLUSTER_FILTERS)
    snapshots = response.get('Snapshots', [])
    if snapshots:
        sorted_snapshots = sorted(snapshots, key=itemgetter('StartTime'), reverse=True)
        snapshot_id = sorted_snapshots[0]['SnapshotId']
        print(snapshot_id)

snap-0f39349a1322cc9d3


In [15]:
assert volume_id or snapshot_id

In [16]:
if not volume_id:
    response = ec2_client.describe_availability_zones()
    availability_zones = response['AvailabilityZones']
    availability_zone = availability_zones[0]['ZoneName']
    # availability_zone = f'{REGION}a'
    if snapshots:
        # TODO: Change this to the latest snapshot!!
        response = ec2_client.create_volume(
            SnapshotId=snapshot_id,
            Size=EBS_VOLUME_SIZE,
            AvailabilityZone=availability_zone,
            VolumeType='gp3',
            TagSpecifications=[
                {
                    'ResourceType': 'volume',
                    'Tags': [{'Key': k, 'Value': TAGS[k]} for k in TAGS]
                }
            ]
        )
    else:
        response = ec2_client.create_volume(
            Size=EBS_VOLUME_SIZE,
            AvailabilityZone=availability_zones[0]['ZoneName'],
            VolumeType='gp3',
            TagSpecifications=[
                {
                    'ResourceType': 'volume',
                    'Tags': [{'Key': k, 'Value': TAGS[k]} for k in TAGS]
                }
            ]
        )
    wait_until(ec2_client.describe_volumes, {'VolumeIds': [response['VolumeId']]}, is_volume_available)
    volume_id = response['VolumeId']
volume_id, availability_zone

'available'

('vol-0111a7543a373b3b9', 'ca-central-1a')

In [17]:
# TODO: Change the parts below to use Terraform

# Create VPC

In [18]:
vpc_ids = get_vpcs_ids(ec2_client, TAGS)
vpc_exists = len(vpc_ids) > 0

if len(vpc_ids) > 1:
    raise RuntimeError
elif vpc_exists:
    vpc_id = vpc_ids[0]

# Create VPC
if not vpc_exists:
    print('Creating VPC...')
    vpc_response = ec2_client.create_vpc(
        CidrBlock='10.0.0.0/16',
        TagSpecifications=[
            {
                'ResourceType': 'vpc',
                'Tags': [{'Key': tag, 'Value': TAGS[tag] }for tag in TAGS]
            }
        ]
    )
    vpc_id = vpc_response['Vpc']['VpcId']
wait_until(ec2_client.describe_vpcs, {'VpcIds': [vpc_id]}, check_all_vpcs_available)

# Create Internet Gateway
igw_ids = get_internet_gateway_ids_attached_to_vpc(ec2_client, vpc_id)
igw_exists = len(igw_ids) > 0

if len(igw_ids) > 1:
    raise RuntimeError
elif igw_exists:
    igw_id = igw_ids[0]


if not igw_exists:
    print('Creating Internet Gateway...')
    # Create an Internet Gateway
    igw_response = ec2_client.create_internet_gateway()
    igw_id = igw_response['InternetGateway']['InternetGatewayId']
    
    # Attach Internet Gateway to VPC
    ec2_client.attach_internet_gateway(
        InternetGatewayId=igw_id,
        VpcId=vpc_id
    )

# Create a route table
route_table_ids = get_route_table_ids_for_vpc(ec2_client, vpc_id)
route_table_exists = len(route_table_ids) > 0

if not route_table_exists:
    print('Creating a route table...')
    # Create a route table
    route_table_response = ec2_client.create_route_table(VpcId=vpc_id)
    route_table_id = route_table_response['RouteTable']['RouteTableId']

is_route_created = False
for route_table_id in route_table_ids:
    if route_to_gateway_exists(ec2_client, route_table_id, igw_id):
        is_route_created = True
        break

if not is_route_created:
    # Create a route to the Internet Gateway
    ec2_client.create_route(
        RouteTableId=route_table_id,
        DestinationCidrBlock='0.0.0.0/0',
        GatewayId=igw_id
    )

vpc_id, igw_id, route_table_id

Creating VPC...
Creating Internet Gateway...


('vpc-0cf9b38d0fcd171c1', 'igw-0efb8fc45d440dc00', 'rtb-0c9abc976bd4d8b81')

# Create Subnets

In [19]:
subnet_ids = get_subnet_ids_in_vpc(ec2_client, vpc_id)
min_subnet_count = 2
current_subnet_count = len(subnet_ids)

# if current_subnet_count < min_subnet_count:
    # response = ec2_client.describe_availability_zones()
    
    # availability_zones = [az['ZoneName'] for az in response['AvailabilityZones']]
    # random.shuffle(availability_zones)
    # TODO: Following logic is still necessary for situations with larger subnets. Change to something reasonable.
    # availability_zones *= ceil(min_subnet_count / len(availability_zones))
    

while current_subnet_count < min_subnet_count:
    i = current_subnet_count + 1
    print(f'Creating subnet #{i}...')
    subnet_response = ec2_client.create_subnet(
        VpcId=vpc_id,
        CidrBlock=f'10.0.{i}.0/24',
        # AvailabilityZone=availability_zone,
        AvailabilityZone=availability_zones[current_subnet_count]['ZoneName'],
        TagSpecifications=[
            {
                'ResourceType': 'subnet',
                'Tags': [{'Key': tag, 'Value': TAGS[tag]} for tag in TAGS]
            }
        ]
    )
    subnet_id = subnet_response['Subnet']['SubnetId']
    subnet_ids.append(subnet_id)
    current_subnet_count = len(subnet_ids)

# check if any subnet is public, add the table to first subnet if not

Creating subnet #1...
Creating subnet #2...


In [20]:
subnets_response = ec2_client.describe_subnets(SubnetIds=subnet_ids)
assert len(subnets_response) == len(subnet_ids) == min_subnet_count
is_any_subnet_open_to_public = False
for subnet in subnets_response['Subnets']:
    if subnets_response.get('MapPublicIpOnLaunch', False):
        # I could also look at the route tables, and see if they are assigned to any subnet. ec2_client.describe_route_tables(RouteTableIds=[route_table_id]) RouteTables.SubnetId (optional parameter)
        public_subnet_id = subnet['SubnetId']
        is_any_subnet_open_to_public = True

if not is_any_subnet_open_to_public:
    for subnet in subnets_response['Subnets']:
        if subnet['CidrBlock'] == '10.0.1.0/24':
            public_subnet_id = subnet['SubnetId']

            # Associate the public subnet with the route table
            ec2_client.associate_route_table(
                RouteTableId=route_table_id,
                SubnetId=public_subnet_id
            )
            
            # Modify the public subnet to auto-assign public IPs
            ec2_client.modify_subnet_attribute(
                SubnetId=public_subnet_id,
                MapPublicIpOnLaunch={"Value": True}
            )
            break
            

# Create Security Group

In [21]:
security_group_ids = get_security_group_ids(ec2_client, TAGS)

security_group_exists = len(security_group_ids) > 0

if len(security_group_ids) > 1:
    raise RuntimeError
elif security_group_exists:
    security_group_id = security_group_ids[0]

# The security group gets torn down when deleted, so there is no need to check the rules and rewrite all of them.

if not security_group_exists:
    print('Creating Security Group...')
    response = ec2_client.create_security_group(
        GroupName=f'eks-cluster-sg-{CLUSTER_NAME}',
        TagSpecifications=[
            {
                'ResourceType': 'security-group', 
                'Tags': [{'Key': tag, 'Value': TAGS[tag]} for tag in TAGS]
            }
        ],
        Description=f'Security group for EKS cluster: {CLUSTER_NAME}',
        VpcId=vpc_id
    )
    security_group_id = response['GroupId']

    # TODO: Can I make the CidrIp more restrictive for the next deployment? Load Balancer needs to have a static IP, probably through the Kubernetes YAML?
    for port in ALLOWED_PORTS:
        ec2_client.authorize_security_group_ingress(
            GroupId=security_group_id,
            IpPermissions=[
                {
                    'IpProtocol': 'tcp',
                    'FromPort': port,
                    'ToPort': port,
                    'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
                }
            ]
        )
    
security_group_id

Creating Security Group...


'sg-0b6c0fb8f72fe6aa2'

In [22]:
volume_id, vpc_id, igw_id, route_table_id, public_subnet_id, subnet_ids, security_group_id

('vol-0111a7543a373b3b9',
 'vpc-0cf9b38d0fcd171c1',
 'igw-0efb8fc45d440dc00',
 'rtb-0c9abc976bd4d8b81',
 'subnet-038f12546d6f50a3c',
 ['subnet-038f12546d6f50a3c', 'subnet-02d016cf23e121bbb'],
 'sg-0b6c0fb8f72fe6aa2')

# Create Cluster

In [23]:
tear_down_and_rebuild = False
response = eks_client.list_clusters()
clusters = response['clusters']
if CLUSTER_NAME not in clusters:
    eks_client.create_cluster(name=CLUSTER_NAME, 
                              version=K8S_VERSION, 
                              roleArn=f'arn:aws:iam::{aws_account_id}:role/EKS_Cluster_Role', 
                              resourcesVpcConfig={'subnetIds': subnet_ids,
                                                  'securityGroupIds': [security_group_id],
                                                  'endpointPublicAccess': True,
                                                  'endpointPrivateAccess': False
                              },
                              tags=TAGS,
                             )
else:
    if tear_down_and_rebuild:
        pass
        # TODO: Create a new cluster with a temp name.
        # Wait until the cluster has finished forming
        # Delete the old cluster
        # Wait until deletion is complete
        # Rename the new cluster
        
wait_until(eks_client.describe_cluster, {'name': CLUSTER_NAME}, is_cluster_active, timeout=7 * 60)
response = eks_client.describe_cluster(name=CLUSTER_NAME)
assert response['cluster']['status'] == 'ACTIVE'

'ACTIVE'

# Create IAM Role for Node Groups

In [24]:
# Define the role name
role_name = 'EKS_Cluster_Role'  # WARNING: I get a weird error if I call this role anything else. The call looks for this particular name, and I do not know how to override it.

# Create the trust policy for the role
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": [
                    "eks.amazonaws.com",
                    "ec2.amazonaws.com"
                ]
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Create the IAM role
try:
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Role for EKS Node Group'
    )
    node_role_arn = response['Role']['Arn']
    print(f"Created role: {node_role_arn}")
except iam_client.exceptions.EntityAlreadyExistsException:
    response = iam_client.get_role(RoleName=role_name)
    node_role_arn = response['Role']['Arn']
    print(f"Role {role_name} already exists. Arn: {node_role_arn}.")
    # TODO: Check if trust_policy is correct.

# Attach necessary policies
policies = [
    'AmazonEKSWorkerNodePolicy',
    'AmazonEC2ContainerRegistryReadOnly',
    'AmazonEKS_CNI_Policy',
    'AmazonEKSClusterPolicy',
    # 'AmazonSSMManagedInstanceCore',
]
# Policy AmazonSSMManagedInstanceCore is not necessary, I used it for debugging, to connect to the node and run commands. 

for policy in policies:
    try:
        iam_client.attach_role_policy(
            RoleName=role_name,
            PolicyArn=f'arn:aws:iam::aws:policy/{policy}'
        )
        print(f"Attached policy {policy} to role {role_name}.")
    except Exception as e:
        print(f"Error attaching policy {policy}: {e}")

Role EKS_Cluster_Role already exists. Arn: arn:aws:iam::275678099358:role/EKS_Cluster_Role.
Attached policy AmazonEKSWorkerNodePolicy to role EKS_Cluster_Role.
Attached policy AmazonEC2ContainerRegistryReadOnly to role EKS_Cluster_Role.
Attached policy AmazonEKS_CNI_Policy to role EKS_Cluster_Role.
Attached policy AmazonEKSClusterPolicy to role EKS_Cluster_Role.


# Add Node Groups

In [25]:
# TODO: This part may also need a wait.

In [26]:
node_groups = eks_client.list_nodegroups(clusterName=CLUSTER_NAME)
# TODO: Make this static and put to the top, with different `diskSize` and `scalingConfig`
node_group_name = 'gpu'
ami_type = 'AL2_x86_64_GPU'
if node_group_name in node_groups['nodegroups']:
    response = eks_client.describe_nodegroup(clusterName=CLUSTER_NAME, nodegroupName=node_group_name)
    status = response['nodegroup']['status']
    if status == 'CREATE_FAILED':
        print('Node group exists, but failed to create. Deleting...')
        failed_node_group_info = response

        eks_client.delete_nodegroup(clusterName=CLUSTER_NAME, nodegroupName=node_group_name)
    
        wait_until(eks_client.list_nodegroups, {'clusterName': CLUSTER_NAME}, lambda x: node_group_name not in x['nodegroups'])
        node_groups = eks_client.list_nodegroups(clusterName=CLUSTER_NAME)
    else:
        print('Node group exists.')



if not node_groups['nodegroups']:
    print('Creating node group...')
    response = eks_client.create_nodegroup(
        clusterName=CLUSTER_NAME,
        nodegroupName=node_group_name,
        scalingConfig={
            'desiredSize': 1,
            'minSize': 1,
            'maxSize': 1
        },
        diskSize=50,  # Size in GiB
        subnets=[public_subnet_id],
        nodeRole=node_role_arn,
        amiType=ami_type,
        instanceTypes=INSTANCE_TYPES[node_group_name],
        labels={
            'gpu-memory': 'true'
        },
        taints=[
            {
                'key': 'nvidia.com/gpu',
                'value': 'true',
                'effect': 'NO_SCHEDULE'
            }
        ]
    )
wait_until(eks_client.describe_nodegroup, {'clusterName': CLUSTER_NAME, 'nodegroupName': node_group_name}, is_node_group_active)

'CREATING'

False

In [27]:
node_groups = eks_client.list_nodegroups(clusterName=CLUSTER_NAME)
# TODO: Make this static and put to the top, with different `diskSize` and `scalingConfig`
node_group_name = 'default'
ami_type = 'AL2_x86_64'
if node_group_name in node_groups['nodegroups']:
    response = eks_client.describe_nodegroup(clusterName=CLUSTER_NAME, nodegroupName=node_group_name)
    status = response['nodegroup']['status']
    if status == 'CREATE_FAILED':
        print('Node group exists, but failed to create. Deleting...')
        failed_node_group_info = response

        eks_client.delete_nodegroup(clusterName=CLUSTER_NAME, nodegroupName=node_group_name)
    
        wait_until(eks_client.list_nodegroups, {'clusterName': CLUSTER_NAME}, lambda x: node_group_name not in x['nodegroups'])
        node_groups = eks_client.list_nodegroups(clusterName=CLUSTER_NAME)
    else:
        print('Node group exists.')


# TODO: See if lower disksize will work.
if node_group_name not in node_groups['nodegroups']:
    print('Creating node group...')
    response = eks_client.create_nodegroup(
        clusterName=CLUSTER_NAME,
        nodegroupName=node_group_name,
        scalingConfig={
            'desiredSize': 2,
            'minSize': 2,
            'maxSize': 2
        },
        diskSize=1000,  # Size in GiB
        subnets=[public_subnet_id],
        nodeRole=node_role_arn,
        amiType=ami_type,
        instanceTypes=INSTANCE_TYPES[node_group_name],
    )
wait_until(eks_client.describe_nodegroup, {'clusterName': CLUSTER_NAME, 'nodegroupName': node_group_name}, is_node_group_active)

'CREATING'

False

# Initialize Kubernetes

In [28]:
!aws eks update-kubeconfig --name $CLUSTER_NAME --region $REGION

Updated context arn:aws:eks:ca-central-1:275678099358:cluster/kubyterlab-llm in /root/.kube/config


# Initialize Helm

In [29]:
!helm repo add aws-ebs-csi-driver https://kubernetes-sigs.github.io/aws-ebs-csi-driver

"aws-ebs-csi-driver" already exists with the same configuration, skipping


In [30]:
!helm repo update

Hang tight while we grab the latest from your chart repositories...
...Successfully got an update from the "aws-ebs-csi-driver" chart repository
Update Complete. ⎈Happy Helming!⎈


# Create a Role to Install the CSI Driver and Give Permissions Using the OIDC Provider

In [31]:
cluster_response = eks_client.describe_cluster(name=CLUSTER_NAME)
oidc_id = cluster_response['cluster']['identity']['oidc']['issuer'].split('/')[-1]
oidc_url = f'https://oidc.eks.{REGION}.amazonaws.com/id/{oidc_id}'
oidc_url

'https://oidc.eks.ca-central-1.amazonaws.com/id/3607A6B25018F663123A7078ED5F5B9E'

In [32]:
!eksctl utils associate-iam-oidc-provider --cluster $CLUSTER_NAME --approve

2025-02-04 22:28:47 [ℹ]  will create IAM Open ID Connect provider for cluster "kubyterlab-llm" in "ca-central-1"
2025-02-04 22:28:47 [✔]  created IAM Open ID Connect provider for cluster "kubyterlab-llm" in "ca-central-1"


In [33]:
amazon_eks_ebs_csi_driver_policy_name = 'AmazonEKS_EBS_CSI_Driver_Policy'
amazon_eks_ebs_csi_driver_policy = {
  "Version": "2012-10-17",
  "Statement": [
    {
      "Effect": "Allow",
      "Action": [
        "ec2:CreateSnapshot",
        "ec2:AttachVolume",
        "ec2:DetachVolume",
        "ec2:ModifyVolume",
        "ec2:DescribeAvailabilityZones",
        "ec2:DescribeInstances",
        "ec2:DescribeSnapshots",
        "ec2:DescribeTags",
        "ec2:DescribeVolumes",
        "ec2:DescribeVolumesModifications"
      ],
      "Resource": "*"
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:CreateTags"
      ],
      "Resource": [
        "arn:aws:ec2:*:*:volume/*",
        "arn:aws:ec2:*:*:snapshot/*"
      ],
      "Condition": {
        "StringEquals": {
          "ec2:CreateAction": [
            "CreateVolume",
            "CreateSnapshot"
          ]
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteTags"
      ],
      "Resource": [
        "arn:aws:ec2:*:*:volume/*",
        "arn:aws:ec2:*:*:snapshot/*"
      ]
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:CreateVolume"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "aws:RequestTag/ebs.csi.aws.com/cluster": "true"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:CreateVolume"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "aws:RequestTag/CSIVolumeName": "*"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteVolume"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "ec2:ResourceTag/CSIVolumeName": "*"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteVolume"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "ec2:ResourceTag/ebs.csi.aws.com/cluster": "true"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteSnapshot"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "ec2:ResourceTag/CSIVolumeSnapshotName": "*"
        }
      }
    },
    {
      "Effect": "Allow",
      "Action": [
        "ec2:DeleteSnapshot"
      ],
      "Resource": "*",
      "Condition": {
        "StringLike": {
          "ec2:ResourceTag/ebs.csi.aws.com/cluster": "true"
        }
      }
    }
  ]
}
try:
    response = iam_client.create_policy(
        PolicyName=amazon_eks_ebs_csi_driver_policy_name,
        PolicyDocument=json.dumps(amazon_eks_ebs_csi_driver_policy)
    )
    amazon_eks_ebs_csi_driver_policy_arn = response['Policy']['Arn']
except iam_client.exceptions.EntityAlreadyExistsException:
    amazon_eks_ebs_csi_driver_policy_arn = f'arn:aws:iam::{aws_account_id}:policy/{amazon_eks_ebs_csi_driver_policy_name}'
    response = iam_client.get_policy(PolicyArn=amazon_eks_ebs_csi_driver_policy_arn)

    # TODO: If policy body is not the same, update. Code to update follows.
    # iam_client.create_policy_version(
    #     PolicyArn=policy_arn,
    #     PolicyDocument=policy_document_json,
    #     SetAsDefault=True
    # )

amazon_eks_ebs_csi_driver_policy_arn

'arn:aws:iam::275678099358:policy/AmazonEKS_EBS_CSI_Driver_Policy'

In [34]:
trust_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Federated": f"arn:aws:iam::{aws_account_id}:oidc-provider/oidc.eks.{REGION}.amazonaws.com/id/{oidc_id}"
            },
            "Action": "sts:AssumeRoleWithWebIdentity",
            "Condition": {
                "StringEquals": {
                    f"oidc.eks.{REGION}.amazonaws.com/id/{oidc_id}:aud": "sts.amazonaws.com",
                    f"oidc.eks.{REGION}.amazonaws.com/id/{oidc_id}:sub": "system:serviceaccount:kube-system:ebs-csi-controller-sa"
                }
            }
        }
    ]
}

try:
    create_role_response = iam_client.create_role(
        RoleName='AmazonEKS_EBS_CSI_DriverRole',
        
        AssumeRolePolicyDocument=json.dumps(trust_policy),
        Description='Role for Amazon EKS EBS CSI Driver'
    )
    print(f"Role created successfully: {create_role_response['Role']['Arn']}")
except iam_client.exceptions.EntityAlreadyExistsException:
    print("Role already exists. Updating the trust policy.")
    iam_client.update_assume_role_policy(
        RoleName='AmazonEKS_EBS_CSI_DriverRole',
        PolicyDocument=json.dumps(trust_policy)
    )

attach_policy_response = iam_client.attach_role_policy(
    RoleName='AmazonEKS_EBS_CSI_DriverRole',
    PolicyArn='arn:aws:iam::aws:policy/service-role/AmazonEBSCSIDriverPolicy'
)

attach_policy_response = iam_client.attach_role_policy(
    RoleName='AmazonEKS_EBS_CSI_DriverRole',
    PolicyArn=amazon_eks_ebs_csi_driver_policy_arn
)

Role already exists. Updating the trust policy.


# Install CSI Driver

In [35]:
!kubectl create sa ebs-csi-controller-sa -n kube-system

serviceaccount/ebs-csi-controller-sa created


In [36]:
!kubectl annotate sa ebs-csi-controller-sa -n kube-system eks.amazonaws.com/role-arn=$ebs_csi_driver_role_arn


serviceaccount/ebs-csi-controller-sa annotated


In [37]:
!kubectl annotate sa ebs-csi-controller-sa -n kube-system meta.helm.sh/release-namespace=kube-system

serviceaccount/ebs-csi-controller-sa annotated


In [38]:
!kubectl annotate sa ebs-csi-controller-sa -n kube-system meta.helm.sh/release-name=aws-ebs-csi-driver

serviceaccount/ebs-csi-controller-sa annotated


In [39]:
!kubectl label sa ebs-csi-controller-sa -n kube-system app.kubernetes.io/managed-by=Helm

serviceaccount/ebs-csi-controller-sa labeled


In [40]:
!helm upgrade --install aws-ebs-csi-driver \
    --namespace kube-system \
    aws-ebs-csi-driver/aws-ebs-csi-driver

Release "aws-ebs-csi-driver" does not exist. Installing it now.
NAME: aws-ebs-csi-driver
LAST DEPLOYED: Tue Feb  4 22:28:54 2025
NAMESPACE: kube-system
STATUS: deployed
REVISION: 1
NOTES:
To verify that aws-ebs-csi-driver has started, run:

    kubectl get pod -n kube-system -l "app.kubernetes.io/name=aws-ebs-csi-driver,app.kubernetes.io/instance=aws-ebs-csi-driver"

[ACTION REQUIRED] Update to the EBS CSI Driver IAM Policy

Due to an upcoming change in handling of IAM polices for the CreateVolume API when creating a volume from an EBS snapshot, a change to your EBS CSI Driver policy may be needed. For more information and remediation steps, see GitHub issue #2190 (https://github.com/kubernetes-sigs/aws-ebs-csi-driver/issues/2190). This change affects all versions of the EBS CSI Driver and action may be required even on clusters where the driver is not upgraded.


### == End of Code Specific to AWS ==

# Install Nvidia Device Plugin

In [41]:
!kubectl apply -f https://raw.githubusercontent.com/NVIDIA/k8s-device-plugin/refs/heads/main/deployments/static/nvidia-device-plugin.yml

daemonset.apps/nvidia-device-plugin-daemonset created


# Apply Kubernetes

In [42]:
# TODO: Change this to Helm Charts

In [43]:
with open(f'/k8s/{CLUSTER_NAME}/{CLUSTER_NAME}-pv.yaml') as f:
    pv = yaml.safe_load(f)
    pv['spec']['awsElasticBlockStore']['volumeID'] = volume_id
    with NamedTemporaryFile('w', suffix='.json', delete=False) as temp_file:
        json.dump(pv, temp_file)
        file_path = temp_file.name
    result = subprocess.run(['kubectl', 'apply', '-f', file_path], capture_output=True, text=True)
result.stdout, result.stderr

('persistentvolume/pv-llm created\n', '')

In [44]:
!kubectl apply -f /k8s/$CLUSTER_NAME/$CLUSTER_NAME-pvc.yaml

persistentvolumeclaim/pvc-llm created


In [45]:
with open(f'/k8s/{CLUSTER_NAME}/{CLUSTER_NAME}-pod.yaml') as f:
    pod = yaml.safe_load(f)
    for i, container in enumerate(pod['spec']['template']['spec']['containers']):
        pod['spec']['template']['spec']['containers'][i]['image'] = container['image']\
			.replace('<AWS_ACCOUNT_ID>', aws_account_id)\
			.replace('<REGION>', REGION)
    with NamedTemporaryFile('w', suffix='.json', delete=False) as temp_file:
        json.dump(pod, temp_file)
        file_path = temp_file.name
    result = subprocess.run(['kubectl', 'apply', '-f', file_path], capture_output=True, text=True)
result.stdout, result.stderr


('deployment.apps/kubyterlab-llm-pod created\n', '')

In [46]:
!kubectl apply -f /k8s/$CLUSTER_NAME/kubyterlab-llm-service.yaml


service/kubyterlab-llm-service created


# Get URL

In [47]:
wait_until(get_one_running_pod, {}, lambda x: x is not None, timeout=8 * 60)
pod_name = get_one_running_pod()
assert pod_name is not None
pod_name

'kubyterlab-llm-pod-85d468557b-cdvcf'

In [48]:
output = !kubectl get service kubyterlab-llm-service --no-headers
fields = re.split(r'\s+', output[0])
external_url = fields[3]
port = fields[4].split('/')[0].split(':')[0]
external_url, port

('a55cfe529cb114c519933d9d0e83a5c4-1462697016.ca-central-1.elb.amazonaws.com',
 '8888')

In [49]:
wait_until(get_jupyter_token_from_pod, {"pod_name": pod_name}, lambda x: x is not None)
token = get_jupyter_token_from_pod(pod_name)
assert token is not None
token

'c77a61fd61c59473542e7cf6800dbdb7aa099f50727b3e4b'

In [50]:
f'http://{external_url}:{port}/lab?token={token}'

'http://a55cfe529cb114c519933d9d0e83a5c4-1462697016.ca-central-1.elb.amazonaws.com:8888/lab?token=c77a61fd61c59473542e7cf6800dbdb7aa099f50727b3e4b'

# Test

In [47]:
# Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
# True
# 1
!kubectl exec $pod_name bash -- python3 -c 'import torch; print(torch.cuda.is_available()); print(torch.cuda.device_count())'

Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
True
1


In [48]:
!kubectl exec $pod_name bash -- bash -c 'nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv'

Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 1 MiB, 15095 MiB


# == End of Procedure ==

## == Do Not Continue: rest of the code is only for reference for troubleshooting ==

# Troubleshoot

In [ ]:
# Note that for the following to work, you need to attach the policy AmazonSSMManagedInstanceCore, in the relevant cell above, during the deployment.

In [60]:
output = !aws ssm send-command \
    --document-name "AWS-RunShellScript" \
    --targets "Key=instanceIds,Values=i-02f3c1f8a118eb352" \
    --parameters 'commands=["nvidia-smi --query-gpu=memory.total,memory.used,memory.free --format=csv"]' \
    --region $REGION
command_id = json.loads(''.join(output))['Command']['CommandId']
command_id


'9528f9bd-ba96-49a8-b6f9-45df5433a905'

In [61]:
output = !aws ssm list-command-invocations \
    --command-id $command_id \
    --details
output = json.loads(''.join(output))['CommandInvocations'][0]['CommandPlugins'][0]['Output']
for line in output.split('\n'):
    print(line)

memory.total [MiB], memory.used [MiB], memory.free [MiB]
15360 MiB, 1 MiB, 15095 MiB



In [31]:
!kubectl get pod -n kube-system -l "app.kubernetes.io/name=aws-ebs-csi-driver,app.kubernetes.io/instance=aws-ebs-csi-driver"

NAME                                 READY   STATUS    RESTARTS   AGE
ebs-csi-controller-59b6797bf-mdw4c   5/5     Running   0          157m
ebs-csi-controller-59b6797bf-psqv5   5/5     Running   0          157m
ebs-csi-node-mxkzh                   3/3     Running   0          157m
ebs-csi-node-nkgzs                   3/3     Running   0          157m
ebs-csi-node-vkj5h                   3/3     Running   0          157m


In [29]:
# NAME                                          GPU
# ip-10-0-1-129.ca-central-1.compute.internal   <none>
# ip-10-0-1-223.ca-central-1.compute.internal   <none>
# ip-10-0-1-8.ca-central-1.compute.internal     1z
!kubectl get nodes "-o=custom-columns=NAME:.metadata.name,GPU:.status.allocatable.nvidia\.com/gpu"


NAME                                          GPU
ip-10-0-1-121.ca-central-1.compute.internal   1
ip-10-0-1-224.ca-central-1.compute.internal   <none>
ip-10-0-1-70.ca-central-1.compute.internal    <none>


In [49]:
# NAME                                          TAINTS
# ip-10-0-1-107.ca-central-1.compute.internal   <none>
# ip-10-0-1-175.ca-central-1.compute.internal   [map[effect:NoSchedule key:nvidia.com/gpu value:true]]
# ip-10-0-1-90.ca-central-1.compute.internal    <none>
!kubectl get nodes -o custom-columns=NAME:.metadata.name,TAINTS:.spec.taints

NAME                                          TAINTS
ip-10-0-1-103.ca-central-1.compute.internal   <none>
ip-10-0-1-167.ca-central-1.compute.internal   [map[effect:NoSchedule key:nvidia.com/gpu value:true]]
ip-10-0-1-251.ca-central-1.compute.internal   <none>


In [50]:
# NAME                     TYPE           CLUSTER-IP     EXTERNAL-IP                                                                  PORT(S)          AGE
# kubernetes               ClusterIP      172.20.0.1     <none>                                                                       443/TCP          37m
# kubyterlab-llm-service   LoadBalancer   172.20.129.8   a4740e3e56bfe40ac81121bd46071903-1377611187.ca-central-1.elb.amazonaws.com   8888:31434/TCP   13s
!kubectl get service

NAME                     TYPE           CLUSTER-IP      EXTERNAL-IP                                                                  PORT(S)          AGE
kubernetes               ClusterIP      172.20.0.1      <none>                                                                       443/TCP          10m
kubyterlab-llm-service   LoadBalancer   172.20.31.136   ad6bb4a45883c4a1399cb117fa0c1814-1996712325.ca-central-1.elb.amazonaws.com   8888:31969/TCP   5m24s


In [286]:
# NAME               READY   UP-TO-DATE   AVAILABLE   AGE
# kubyterlab-llm-pod   1/1     1            1           31m
!kubectl get deployments

NAME                 READY   UP-TO-DATE   AVAILABLE   AGE
kubyterlab-llm-pod   0/1     1            0           54m


In [252]:
# !kubectl describe deployment kubyterlab-llm-pod

In [83]:
!kubectl get pods

NAME                                  READY   STATUS    RESTARTS       AGE
kubyterlab-llm-pod-5b75b56d56-rpqj9   1/1     Running   2 (116s ago)   79m


In [254]:
!kubectl get pvc

NAME      STATUS   VOLUME   CAPACITY   ACCESS MODES   STORAGECLASS   VOLUMEATTRIBUTESCLASS   AGE
pvc-llm   Bound    pv-llm   500Gi      RWO            manual         <unset>                 26m


In [ ]:
!kubectl exec $pod_name bash -- nvidia-smi --version

In [63]:
!kubectl exec $pod_name bash -- nvidia-smi

Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
Mon Jan 13 22:15:38 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       On  |   00000000:00:1E.0 Off |                    0 |
| N/A   25C    P8             11W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |              

In [62]:
!kubectl exec $pod_name bash -- nvidia-smi -L

Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
GPU 0: Tesla T4 (UUID: GPU-fe1acad6-41f9-c441-44ed-d4ead6db6dc2)


In [61]:
!kubectl exec $pod_name bash -- nvcc --version

Defaulted container "kubyterlab-llm" out of: kubyterlab-llm, init-container (init)
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Aug_14_10:10:22_PDT_2024
Cuda compilation tools, release 12.6, V12.6.68
Build cuda_12.6.r12.6/compiler.34714021_0


In [ ]:
!kubectl describe pod $pod_name

In [113]:
!kubectl get pod -n kube-system

NAME                                   READY   STATUS    RESTARTS   AGE
aws-node-cvzp2                         2/2     Running   0          4m59s
aws-node-kmnjj                         2/2     Running   0          4m57s
aws-node-zlmn8                         2/2     Running   0          6m35s
coredns-749d5dbdd9-cdd8v               1/1     Running   0          8m29s
coredns-749d5dbdd9-xtrc8               1/1     Running   0          8m29s
ebs-csi-controller-59b6797bf-fhcph     5/5     Running   0          4m6s
ebs-csi-controller-59b6797bf-w9mns     5/5     Running   0          4m6s
ebs-csi-node-mp9sr                     3/3     Running   0          4m6s
ebs-csi-node-mqt55                     3/3     Running   0          4m6s
ebs-csi-node-q8dfl                     3/3     Running   0          4m6s
kube-proxy-ctjcd                       1/1     Running   0          6m35s
kube-proxy-nv8tx                       1/1     Running   0          4m57s
kube-proxy-qmwmh                       1/1   

In [177]:
!kubectl describe pvc pvc-llm

Name:          pvc-llm
Namespace:     default
StorageClass:  manual
Status:        Bound
Volume:        pv-llm
Labels:        <none>
Annotations:   pv.kubernetes.io/bind-completed: yes
               pv.kubernetes.io/bound-by-controller: yes
Finalizers:    [kubernetes.io/pvc-protection]
Capacity:      10Gi
Access Modes:  RWO
VolumeMode:    Filesystem
Used By:       kubyterlab-llm-pod-67f5cf95dc-s2mrn
Events:        <none>


In [178]:
!kubectl describe pv pv-llm

Name:            pv-llm
Labels:          <none>
Annotations:     pv.kubernetes.io/bound-by-controller: yes
Finalizers:      [kubernetes.io/pv-protection]
StorageClass:    manual
Status:          Bound
Claim:           default/pvc-llm
Reclaim Policy:  Retain
Access Modes:    RWO
VolumeMode:      Filesystem
Capacity:        10Gi
Node Affinity:   <none>
Message:         
Source:
    Type:       AWSElasticBlockStore (a Persistent Disk resource in AWS)
    VolumeID:   vol-04ce08a2c05b8e1da
    FSType:     ext4
    Partition:  0
    ReadOnly:   false
Events:         <none>


In [169]:
!kubectl describe deployment kubyterlab-llm-pod

Name:                   kubyterlab-llm-pod
Namespace:              default
CreationTimestamp:      Mon, 02 Dec 2024 20:05:56 +0000
Labels:                 <none>
Annotations:            deployment.kubernetes.io/revision: 1
Selector:               app=kubyterlab-llm
Replicas:               1 desired | 1 updated | 1 total | 0 available | 1 unavailable
StrategyType:           RollingUpdate
MinReadySeconds:        0
RollingUpdateStrategy:  25% max unavailable, 25% max surge
Pod Template:
  Labels:  app=kubyterlab-llm
  Containers:
   kubyterlab-llm:
    Image:      kubyterlab:24.10
    Port:       8888/TCP
    Host Port:  0/TCP
    Limits:
      cpu:             1
      memory:          8Gi
      nvidia.com/gpu:  1
    Requests:
      cpu:             1
      memory:          8Gi
      nvidia.com/gpu:  1
    Environment:
      JUPYTERLAB_SETTINGS_DIR:  /jupyterlab/config
      MISTRAL_MODEL:            /models/mistral
    Mounts:
      /corpus from pv-llm (rw)
      /jupyterlab from pv-llm

In [59]:
!kubectl get nodes

NAME                                          STATUS   ROLES    AGE   VERSION
ip-10-0-1-100.ca-central-1.compute.internal   Ready    <none>   47m   v1.30.7-eks-59bf375
ip-10-0-1-25.ca-central-1.compute.internal    Ready    <none>   47m   v1.30.7-eks-59bf375
ip-10-0-1-50.ca-central-1.compute.internal    Ready    <none>   48m   v1.30.7-eks-59bf375


In [58]:
!kubectl get configmap

NAME               DATA   AGE
kube-root-ca.crt   1      50m


In [57]:
!kubectl get all -l app.kubernetes.io/name=aws-ebs-csi-driver -n kube-system

NAME                                     READY   STATUS    RESTARTS   AGE
pod/ebs-csi-controller-59b6797bf-vfvs2   5/5     Running   0          47m
pod/ebs-csi-controller-59b6797bf-xt7fv   5/5     Running   0          47m
pod/ebs-csi-node-dwzz7                   3/3     Running   0          47m
pod/ebs-csi-node-gplqp                   3/3     Running   0          47m
pod/ebs-csi-node-l5nh6                   3/3     Running   0          47m

NAME                          DESIRED   CURRENT   READY   UP-TO-DATE   AVAILABLE   NODE SELECTOR            AGE
daemonset.apps/ebs-csi-node   3         3         3       3            3           kubernetes.io/os=linux   47m

NAME                                 READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/ebs-csi-controller   2/2     2            2           47m

NAME                                           DESIRED   CURRENT   READY   AGE
replicaset.apps/ebs-csi-controller-59b6797bf   2         2         2       47m


In [56]:
!aws iam list-open-id-connect-providers

{
    "OpenIDConnectProviderList": [
        {
            "Arn": "arn:aws:iam::275678099358:oidc-provider/oidc.eks.ca-central-1.amazonaws.com/id/8D7619520212428BD59C46B20BF19338"
        }
    ]
}


In [51]:
!kubectl logs deployment/ebs-csi-controller -n kube-system -c ebs-plugin

Found 2 pods, using pod/ebs-csi-controller-59b6797bf-vfvs2
I0113 21:28:06.936951       1 main.go:153] "Initializing metadata"
I0113 21:28:06.943481       1 metadata.go:48] "Retrieved metadata from IMDS"
I0113 21:28:06.944585       1 driver.go:69] "Driver Information" Driver="ebs.csi.aws.com" Version="v1.38.1"
I0113 22:01:16.236299       1 controller.go:410] "ControllerPublishVolume: attaching" volumeID="vol-0ed5d3cc8cb5a989e" nodeID="i-0fd8ede93d6ac59a4"
I0113 22:01:17.882498       1 controller.go:419] "ControllerPublishVolume: attached" volumeID="vol-0ed5d3cc8cb5a989e" nodeID="i-0fd8ede93d6ac59a4" devicePath="/dev/xvdaa"


In [52]:
!kubectl logs daemonset/ebs-csi-node -n kube-system -c ebs-plugin

Found 3 pods, using pod/ebs-csi-node-l5nh6
I0113 21:27:55.828513       1 main.go:153] "Initializing metadata"
I0113 21:27:55.832956       1 metadata.go:48] "Retrieved metadata from IMDS"
I0113 21:27:55.833450       1 driver.go:69] "Driver Information" Driver="ebs.csi.aws.com" Version="v1.38.1"
E0113 21:27:56.896438       1 node.go:856] "Unexpected failure when attempting to remove node taint(s)" err="isAllocatableSet: driver not found on node ip-10-0-1-50.ca-central-1.compute.internal"
I0113 21:27:57.410789       1 node.go:936] "CSINode Allocatable value is set" nodeName="ip-10-0-1-50.ca-central-1.compute.internal" count=24
I0113 22:01:18.817732       1 node.go:204] "NodeStageVolume: invalid partition config, will ignore." partition="0"
I0113 22:01:18.980008       1 mount_linux.go:295] Detected OS without systemd


In [53]:
!kubectl logs deployment/ebs-csi-controller -n kube-system -c csi-provisioner

Found 2 pods, using pod/ebs-csi-controller-59b6797bf-vfvs2
W0113 21:28:09.566495       1 feature_gate.go:354] Setting GA feature gate Topology=true. It will be removed in a future release.
I0113 21:28:09.566789       1 feature_gate.go:387] feature gates: {map[Topology:true]}
I0113 21:28:09.566840       1 csi-provisioner.go:154] Version: v5.1.0
I0113 21:28:09.566851       1 csi-provisioner.go:177] Building kube configs for running in cluster...
I0113 21:28:09.568886       1 common.go:143] "Probing CSI driver for readiness"
I0113 21:28:09.598446       1 csi-provisioner.go:230] Detected CSI driver ebs.csi.aws.com
I0113 21:28:09.598619       1 csi-provisioner.go:240] Supports migration from in-tree plugin: kubernetes.io/aws-ebs
I0113 21:28:09.600672       1 common.go:143] "Probing CSI driver for readiness"
I0113 21:28:09.603385       1 csi-provisioner.go:299] CSI driver supports PUBLISH_UNPUBLISH_VOLUME, watching VolumeAttachments
I0113 21:28:09.604750       1 controller.go:744] "Using sav

In [54]:
!kubectl logs deployment/ebs-csi-controller -n kube-system -c csi-attacher

Found 2 pods, using pod/ebs-csi-controller-59b6797bf-vfvs2
I0113 21:28:12.211572       1 main.go:109] "Version" version="v4.7.0"
I0113 21:28:12.215865       1 common.go:143] "Probing CSI driver for readiness"
I0113 21:28:12.238251       1 main.go:169] "CSI driver name" driver="ebs.csi.aws.com"
I0113 21:28:12.240448       1 common.go:143] "Probing CSI driver for readiness"
I0113 21:28:12.242734       1 main.go:249] "CSI driver supports ControllerPublishUnpublish, using real CSI handler" driver="ebs.csi.aws.com"
I0113 21:28:12.243624       1 leaderelection.go:254] attempting to acquire leader lease kube-system/external-attacher-leader-ebs-csi-aws-com...
I0113 21:28:12.265555       1 leaderelection.go:268] successfully acquired lease kube-system/external-attacher-leader-ebs-csi-aws-com
I0113 21:28:12.266787       1 leader_election.go:184] "became leader, starting"
I0113 21:28:12.266811       1 controller.go:129] "Starting CSI attacher"
I0113 21:28:12.267257       1 envvar.go:172] "Feature

In [55]:
!kubectl get sa ebs-csi-controller-sa -n kube-system -o jsonpath='{.metadata.annotations}'

{"eks.amazonaws.com/role-arn":"","meta.helm.sh/release-name":"aws-ebs-csi-driver","meta.helm.sh/release-namespace":"kube-system"}